In [85]:
# Necessary imports to run this notebook.
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.metrics import accuracy_score  # To measure the accuracy score for outputting the metric in the console
from sklearn.metrics.pairwise import euclidean_distances  # To compute Euclidean distances 
from sklearn.model_selection import LeaveOneOut # Using leave one out cross validation 

In [86]:
# Loading the dataset
def load_dataset(file_path):
    data = np.loadtxt(file_path)
    X = data[:, 1:]  # Capturing data from 2nd column since 1st column is containing labels y
    y = data[:, 0].astype(int)  # Assigning labels 
    return X, y

In [87]:
# Implemented this KNearestNeighbors Classifier logic based on this youtube tutorial below:- 
# https://www.youtube.com/watch?v=rTEtEy5o3X0
class KNearestNeighbors:
    def __init__(self, k=3):
        self.k = k

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X_test):
        predictions = []
        for x_test in X_test:
            # Compute the Euclidean distances from x_test to all training samples
            distances = euclidean_distances(x_test.reshape(1, -1), self.X_train).flatten()
            
            # Get the indices of the k nearest neighbors
            k_indices = np.argsort(distances)[:self.k]
            
            # Extract the labels of the k nearest neighbors
            k_nearest_labels = [self.y_train[i] for i in k_indices]
            
            # Use the most common label from the majority vote
            most_common = Counter(k_nearest_labels).most_common(1)[0][0]
            
            predictions.append(most_common)
        return np.array(predictions)

In [88]:
# Code referenced from the MATPLOTLIB code in slide "Project_2_Briefing.pdf"
def leave_one_out_cross_validation(X, y, feature_indices):
    # Initialize the K-Nearest Neighbors classifier with k=3 for LOOCV
    classifier = KNearestNeighbors(k=3)
    
    # Prepare the dataset with only the selected features
    X_subset = X[:, feature_indices]
    
    # Set up the leave-one-out cross-validation
    loo = LeaveOneOut()
    number_correctly_classified = 0
    
    # Iterate over each fold for leave-one-out
    for train_index, test_index in loo.split(X_subset):
        X_train, X_test = X_subset[train_index], X_subset[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Fit the model on the training part
        classifier.fit(X_train, y_train)
        
        # Predict the outcome for the test part
        predicted_label = classifier.predict(X_test)
        
        # Count correct predictions
        if predicted_label == y_test:
            number_correctly_classified += 1
    
    # Calculate and return accuracy as the ratio of correctly classified instances
    accuracy = number_correctly_classified / len(X_subset)
    return accuracy

In [89]:
def feature_selection(X, y, method='forward'):
    n_features = X.shape[1] # fetching the columns amount 
    if method == 'forward': # Code logic referenced from Slide "Project_2_Briefing.pdf" where it was used to implement the logic
        
        # Starting with no features selected 
        current_features = []
        best_accuracy = 0
        
        while len(current_features) < n_features:
            best_current_feature = None
            
            # Adds the each feature to the current set and checks for accuracy
            for feature in range(n_features):
                
                if feature in current_features:
                    continue # Skipping if feature is already selected 
                
                # Temporarily adding features and selecting accuracy 
                temp_features = current_features + [feature] 
                accuracy = leave_one_out_cross_validation(X, y, temp_features)
                print(f"Using feature(s) {[ f + 1 for f in temp_features ]} accuracy is {accuracy*100:.1f}%")
                
                # Remembering the feature if it is improving accuracy 
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_current_feature = feature
                    
            # Adding the feature to current set if it is improving accuracy 
            if best_current_feature is not None:
                current_features.append(best_current_feature)
                print(f"Feature set {[ f + 1 for f in current_features]} was best, accuracy is {best_accuracy*100:.1f}%")
            else:
                break # If no feature is improving the accuracy, we stop the search 
            
        print(f"Finished search!! The best feature subset is {[ f + 1 for f in current_features]}, which has an accuracy of {best_accuracy*100:.1f}%")
        return current_features

    elif method == 'backward':
        
        # Starting with all features selected 
        current_features = list(range(n_features)) # I used forward elimination's code which was derived from  Slide "Project_2_Briefing.pdf" and then built upon it
        while len(current_features) > 1:
            worst_current_feature = None
            best_current_accuracy = 0
            
            # Removes each feature to the current set and checks for accuracy
            for feature in current_features:
                temp_features = current_features[:]
                temp_features.remove(feature)
                accuracy = leave_one_out_cross_validation(X, y, temp_features)
                print(f"Using feature(s) {[ f + 1 for f in temp_features]} accuracy is {accuracy*100:.1f}%")
                
                # If removal of the feature improves accuracy then we are remembering this feature 
                if accuracy > best_current_accuracy:
                    best_current_accuracy = accuracy
                    worst_current_feature = feature
            
            # Finding the feature whose removal improves accuracy, we remove it from the current set
            if worst_current_feature is not None:
                current_features.remove(worst_current_feature)
                print(f"Feature set {[ f + 1 for f in current_features]} was best, accuracy is {best_current_accuracy*100:.1f}%")
            else:
                break # If there was no improvement in accuracy by removing the feature, we stop the search 
        print(f"Finished search!! The best feature subset is {[ f + 1 for f in current_features]}, which has an accuracy of {best_current_accuracy*100:.1f}%")
        return current_features 

In [91]:
file_name = input("Welcome to Rachit Prajapati's Feature Selection Algorithm.\nType in the name of the file to test: ") 
X, y = load_dataset(file_name) # Fetching the file by path and extracting it by data in columns and labels 
print(f"This dataset has {X.shape[1]} features (not including the class attribute), with {X.shape[0]} instances.")

method = input("Type the number of the algorithm you want to run.\n1) Forward Selection\n2) Backward Elimination\n")
method = 'forward' if method == '1' else 'backward' # code to select the respective mode 

selected_features = feature_selection(X, y, method) # Call the search algorithm as per the respective mode selected